In [ ]:
!pip install flask pyngrok transformers sentencepiece torch --quiet

In [ ]:
%%writefile app.py
from flask import Flask, render_template, request
from transformers import MarianMTModel, MarianTokenizer
import functools

app = Flask(__name__)

LANGS = {
    "English": "en",
    "Hindi": "hi",
    "French": "fr",
    "Spanish": "es",
    "German": "de",
    "Italian": "it",
    "Portuguese": "pt",
    "Russian": "ru",
}

# Cache models so they don’t reload every request
@functools.lru_cache(maxsize=20)
def load_model(src, tgt):
    model_name = f"Helsinki-NLP/opus-mt-{src}-{tgt}"
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return model, tokenizer



# model_name = f"Helsinki-NLP/opus-mt-{src}-{tgt}"`

# Builds the HuggingFace model name dynamically. Example:

# Helsinki-NLP/opus-mt-en-hi` → English to Hindi model.


# Main Route (Homepage)
@app.route("/", methods=["GET", "POST"])
def home():
    translation = ""
    if request.method == "POST":
        src_lang = request.form["src_lang"]
        tgt_lang = request.form["tgt_lang"]
        text = request.form["text"]

        try:
            src_code = LANGS[src_lang]
            tgt_code = LANGS[tgt_lang]
            model, tokenizer = load_model(src_code, tgt_code)

            inputs = tokenizer(text, return_tensors="pt", padding=True)
            translated_tokens = model.generate(**inputs, max_length=256)
            translation = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
        except Exception as e:
            translation = f"❌ Error: {e}"

    return render_template("index.html", langs=LANGS.keys(), translation=translation)

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=False)


Writing app.py


In [ ]:
!mkdir -p templates
!mkdir -p static


In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html>
<head>
    <title>🌍 Multilingual Translator</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="container">
        <h1>🌍 Multilingual Translator</h1>
        <form method="post" class="card">
            <label>Source Language:</label>
            <select name="src_lang">
                {% for lang in langs %}
                <option>{{ lang }}</option>
                {% endfor %}
            </select>

            <label>Target Language:</label>
            <select name="tgt_lang">
                {% for lang in langs %}
                <option>{{ lang }}</option>
                {% endfor %}
            </select>

            <textarea name="text" rows="5" placeholder="Enter text here..."></textarea>
            <button type="submit">Translate 🚀</button>
        </form>

        {% if translation %}
        <div class="result">
            <h2>✅ Translation:</h2>
            <p>{{ translation }}</p>
        </div>
        {% endif %}
    </div>
</body>
</html>


Writing templates/index.html


In [ ]:
%%writefile static/style.css
body {
    font-family: 'Segoe UI', sans-serif;
    background: linear-gradient(135deg, #141E30, #243B55);
    color: white;
    display: flex;
    justify-content: center;
    align-items: center;
    height: 100vh;
    margin: 0;
}
.container {
    text-align: center;
    width: 60%;
}
h1 {
    margin-bottom: 20px;
}
.card {
    background: rgba(255, 255, 255, 0.1);
    padding: 20px;
    border-radius: 12px;
    box-shadow: 0 0 15px rgba(0,0,0,0.4);
}
select, textarea, button {
    width: 90%;
    margin: 10px 0;
    padding: 10px;
    border-radius: 8px;
    border: none;
}
button {
    background: #FFD700;
    cursor: pointer;
    font-weight: bold;
}
button:hover {
    background: #FFA500;
}
.result {
    margin-top: 20px;
    padding: 15px;
    background: rgba(255,255,255,0.1);
    border-radius: 10px;
}


Writing static/style.css


In [ ]:
# ✅ Kill any running Flask/ngrok processes
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"


^C
^C


In [ ]:
!lsof -i :8000


In [ ]:
!kill -9 6798  # here ID no shud be changed based on the above cell { PID 1949 }


/bin/bash: line 1: kill: (6798) - No such process


In [ ]:
# ✅ Run Flask in the background and log output
!nohup python app.py > flask.log 2>&1 &


In [ ]:
# ✅ Start ngrok tunnel
from pyngrok import ngrok, conf
conf.get_default().auth_token = "2ztgrogNy3iJnxaz8wchWgCw1X7_4vgQZE6DfL2kHMu2DmQzM"

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# ✅ Check Flask logs (useful if error happens)
!sleep 3 && tail -n 20 flask.log


ERROR:pyngrok.process.ngrok:t=2025-09-10T10:39:45+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2025-09-10T10:39:45+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok st

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.

# Explanation



## 1️⃣ Install Dependencies

```python
!pip install flask pyngrok transformers sentencepiece torch --quiet
```

* `!pip install ...` → installs Python packages inside the Colab environment.
* `flask` → lightweight Python web framework.
* `pyngrok` → makes local web apps publicly accessible by creating a tunnel.
* `transformers` → HuggingFace library for NLP (translation, summarization, etc.).
* `sentencepiece` → tokenizer library (needed by translation models).
* `torch` → PyTorch framework (runs the deep learning models).
* `--quiet` → suppresses extra output during installation.

---

## 2️⃣ Create `app.py` (main Flask backend)

```python
%%writefile app.py
```

* Writes everything below into a new file named `app.py`.

```python
from flask import Flask, render_template, request
```

* Imports Flask:

  * `Flask` → to create the web app.
  * `render_template` → to display HTML files.
  * `request` → to get form data from the user (text & language selection).

```python
from transformers import MarianMTModel, MarianTokenizer
```

* HuggingFace’s MarianMT translation models.
* `MarianMTModel` → the neural network that does the actual translation.
* `MarianTokenizer` → breaks sentences into tokens the model understands.

```python
import functools
```

* Python built-in module. We use it for **caching** to avoid reloading models again and again.

```python
app = Flask(__name__)
```

* Initializes the Flask app.

---

### Language Dictionary

```python
LANGS = {
    "English": "en",
    "Hindi": "hi",
    "French": "fr",
    "Spanish": "es",
    "German": "de",
    "Italian": "it",
    "Portuguese": "pt",
    "Russian": "ru",
}
```

* A mapping between **human-friendly names** and **language codes**.
* Example: `"English": "en"` → the model name will use `en` for English.

---

### Model Loading Function

```python
@functools.lru_cache(maxsize=20)
def load_model(src, tgt):
    model_name = f"Helsinki-NLP/opus-mt-{src}-{tgt}"
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return model, tokenizer
```

* `@functools.lru_cache(maxsize=20)` → caches the last 20 models in memory.
  🔑 Benefit: faster response since model isn’t reloaded every request.

* `model_name = f"Helsinki-NLP/opus-mt-{src}-{tgt}"`
  Builds the HuggingFace model name dynamically. Example:
  `Helsinki-NLP/opus-mt-en-hi` → English to Hindi model.

* `MarianTokenizer.from_pretrained(model_name)` → loads the tokenizer.

* `MarianMTModel.from_pretrained(model_name)` → loads the trained translation model.

* Returns `(model, tokenizer)` pair for use in translation.

---

### Main Route (Homepage)

```python
@app.route("/", methods=["GET", "POST"])
def home():
    translation = ""
```

* Defines the route `/` (homepage).
* Accepts both GET (normal page load) and POST (form submitted).
* `translation = ""` initializes an empty translation.

```python
    if request.method == "POST":
        src_lang = request.form["src_lang"]
        tgt_lang = request.form["tgt_lang"]
        text = request.form["text"]
```

* If the user submits the form:

  * `src_lang` → source language chosen.
  * `tgt_lang` → target language chosen.
  * `text` → text entered by the user.

```python
        try:
            src_code = LANGS[src_lang]
            tgt_code = LANGS[tgt_lang]
            model, tokenizer = load_model(src_code, tgt_code)
```

* Converts user-friendly names (`English`) → short codes (`en`).
* Loads the appropriate model & tokenizer.

```python
            inputs = tokenizer(text, return_tensors="pt", padding=True)
            translated_tokens = model.generate(**inputs, max_length=256)
            translation = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
```

* Tokenizes input text into tensors PyTorch can process.
* `model.generate(...)` → produces translated text tokens.
* `decode(..., skip_special_tokens=True)` → converts tokens back into human-readable text, removing extra tokens like `<pad>`.

```python
        except Exception as e:
            translation = f"❌ Error: {e}"
```

* If something goes wrong (like model missing), shows an error message.

```python
    return render_template("index.html", langs=LANGS.keys(), translation=translation)
```

* Passes two things to `index.html`:

  1. `langs=LANGS.keys()` → dropdown options for languages.
  2. `translation` → the output text.

---

### Flask Runner

```python
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=False)
```

* Runs Flask server on all interfaces (`0.0.0.0`), port `8000`.
* `debug=False` → no debugging messages.

---

## 3️⃣ Create Templates and Static Folders

```bash
!mkdir -p templates
!mkdir -p static
```

* `templates/` → stores HTML files.
* `static/` → stores CSS/JS/images.

---

## 4️⃣ HTML File (`templates/index.html`)

This is what users see in the browser.

```html
<!DOCTYPE html>
<html>
<head>
    <title>🌍 Multilingual Translator</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
```

* Title + links CSS file from `static/style.css`.

```html
    <div class="container">
        <h1>🌍 Multilingual Translator</h1>
        <form method="post" class="card">
```

* Big title and a form (`POST` method).

```html
            <label>Source Language:</label>
            <select name="src_lang">
                {% for lang in langs %}
                <option>{{ lang }}</option>
                {% endfor %}
            </select>
```

* Dropdown for source language.
* `langs` is a list from Flask (`LANGS.keys()`).

```html
            <label>Target Language:</label>
            <select name="tgt_lang">
                {% for lang in langs %}
                <option>{{ lang }}</option>
                {% endfor %}
            </select>
```

* Dropdown for target language.

```html
            <textarea name="text" rows="5" placeholder="Enter text here..."></textarea>
            <button type="submit">Translate 🚀</button>
        </form>
```

* User enters text and clicks "Translate 🚀".

```html
        {% if translation %}
        <div class="result">
            <h2>✅ Translation:</h2>
            <p>{{ translation }}</p>
        </div>
        {% endif %}
```

* If translation exists, display it.

---

## 5️⃣ CSS Styling (`static/style.css`)

* Dark gradient background.
* Centered content.
* Card with rounded edges and shadow.
* Dropdowns, text box, button styled nicely.
* Button hover effect.

---

## 6️⃣ Run Flask + Ngrok

```python
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"
```

* Kill old Flask/ngrok processes.

```python
!nohup python app.py > flask.log 2>&1 &
```

* Run Flask in the background.
* Logs are written to `flask.log`.

```python
from pyngrok import ngrok, conf
conf.get_default().auth_token = "YOUR_AUTH_TOKEN"

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)
```

* Authenticates Ngrok with your token.
* Exposes Flask app running on port 8000 to the internet.
* Gives a **public URL** you can share with students.

```python
!sleep 3 && tail -n 20 flask.log
```

* Waits 3 seconds and prints last 20 lines of Flask log for debugging.

---

✅ **Final Flow to Explain to Students**

1. User opens translator in browser.
2. Chooses source & target language.
3. Enters text → hits **Translate 🚀**.
4. Flask gets request → loads HuggingFace translation model.
5. Model processes text → returns translation.
6. Page displays translated text beautifully.

---


